# utils

> utility stuff.
 



In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
from fastcore.test import *
from fastai.vision.all import *
import torch
from torchvision.models import resnet18, resnet34, resnet50
from typing import Literal
import random 
import os 
import yaml
import numpy as np
import yaml
import configparser
from types import SimpleNamespace
import importlib
from nbdev import config
import json
import hashlib
import subprocess
import re


In [ ]:
#|export
cfg = config.get_config()
PACKAGE_NAME = cfg.lib_name

In [ ]:
#| export
def test_grad_on(model):
    """
    Test that all grads are on for modules with parameters.
    """
    for name, module in model.named_modules():
        # Check each parameter in the module
        for param_name, param in module.named_parameters(recurse=False):
            assert param.requires_grad, f"Gradients are off for {name}.{param_name}"

def test_grad_off(model):
    """
    Test that all non-batch norm grads are off, but batch norm grads are on.
    """
    for name, module in model.named_modules():
        # Distinguish between BatchNorm and other layers
        if isinstance(module, (torch.nn.BatchNorm1d, torch.nn.BatchNorm2d, torch.nn.BatchNorm3d)):
            for param_name, param in module.named_parameters(recurse=False):
                assert param.requires_grad, f"BatchNorm parameter does not require grad in {name}.{param_name}"
        else:
            for param_name, param in module.named_parameters(recurse=False):
                assert not param.requires_grad, f"Gradients are on for non-BatchNorm layer {name}.{param_name}"

In [ ]:
#| export
def seed_everything(seed=42):
    """"
    Seed everything.
    """   
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
#| export

def adjust_config_with_derived_values(config):
    # Adjust n_in based on dataset

    # Adjust encoder_dimension based on architecture
        
    if config.arch == 'smallres':
        config.encoder_dimension = 512
    elif config.arch == 'resnet18':
        config.encoder_dimension = 512
    elif config.arch == 'resnet34':
        config.encoder_dimension = 512
    elif config.arch == 'resnet50':
        config.encoder_dimension = 2048

    else :
        raise ValueError(f"Architecture {config.arch} not supported")

    return config

def load_config(file_path):
    with open(file_path, 'r') as f:
        config = yaml.safe_load(f)
        config = SimpleNamespace(**config)
        config = adjust_config_with_derived_values(config)
        

    return config

In [ ]:
#| export

def get_ssl_dls(dataset,bs,size,device,pct_dataset=1.0):
    # Define the base package name in a variable for easy modification

    try:
        # Construct the module path
        module_path = f"{PACKAGE_NAME}.{dataset}_dataloading"
        
        # Dynamically import the module
        dataloading_module = importlib.import_module(module_path)
    except ModuleNotFoundError:
        # Handle the case where the module cannot be found
        raise ImportError(f"Could not find a data loading module for '{dataset}'. "
                          f"Make sure '{module_path}' exists and is correctly named.") from None
    
    # Assuming the function name follows a consistent naming convention
    func_name = f"get_bt_{dataset}_train_dls"
    try:
        # Retrieve the data loading function from the module
        data_loader_func = getattr(dataloading_module, func_name)
    except AttributeError:
        # Handle the case where the function does not exist in the module
        raise AttributeError(f"The function '{func_name}' was not found in '{module_path}'. "
                             "Ensure it is defined and named correctly.") from None
    
    # Proceed to call the function with arguments from the config
    try:
        dls_train = data_loader_func(bs=bs,size=size,device=device,pct_dataset=pct_dataset)
    except Exception as e:
        # Handle any errors that occur during the function call
        raise RuntimeError(f"An error occurred while calling '{func_name}' from '{module_path}': {e}") from None
    
    return dls_train


In [ ]:
#| export

def get_supervised_dls(dataset,bs,bs_test,size,device):
    "Get train and test dataloaders for supervised learning"

    try:
        # Construct the module path
        module_path = f"{PACKAGE_NAME}.{dataset}_dataloading"
        
        # Dynamically import the module
        dataloading_module = importlib.import_module(module_path)
    except ModuleNotFoundError:
        # Handle the case where the module cannot be found
        raise ImportError(f"Could not find a data loading module for '{dataset}'. "
                          f"Make sure '{module_path}' exists and is correctly named.") from None
    
    # Assuming the function name follows a consistent naming convention
    func_name_train = f"get_supervised_{dataset}_train_dls"
    try:
        # Retrieve the data loading function from the module
        train_data_loader_func = getattr(dataloading_module, func_name_train)
    except AttributeError:
        # Handle the case where the function does not exist in the module
        raise AttributeError(f"The function '{func_name_train}' was not found in '{module_path}'. "
                             "Ensure it is defined and named correctly.") from None
    
    # Proceed to call the function with arguments from the config
    try:
        dls_train = train_data_loader_func(bs=bs,size=size,device=device)
    except Exception as e:
        # Handle any errors that occur during the function call
        raise RuntimeError(f"An error occurred while calling '{func_name_train}' from '{module_path}': {e}") from None
    
    
      # Assuming the function name follows a consistent naming convention
    func_name_test = f"get_supervised_{dataset}_test_dls"
    try:
        # Retrieve the data loading function from the module
        test_data_loader_func = getattr(dataloading_module, func_name_test)
    except AttributeError:
        # Handle the case where the function does not exist in the module
        raise AttributeError(f"The function '{func_name_test}' was not found in '{module_path}'. "
                             "Ensure it is defined and named correctly.") from None
    
    # Proceed to call the function with arguments from the config
    try:
        dls_test = test_data_loader_func(bs=bs_test,size=size,device=device)
    except Exception as e:
        # Handle any errors that occur during the function call
        raise RuntimeError(f"An error occurred while calling '{func_name_test}' from '{module_path}': {e}") from None
    
    

    return {'train_dls':dls_train,'test_dls':dls_test}


    

Basic network mostly used for testing.

In [ ]:
#| export
class _SmallRes(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        # First layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Minimal Intermediate layer
        self.intermediate_conv = nn.Conv2d(64, 512, kernel_size=1, stride=1, bias=False)
        self.intermediate_bn = nn.BatchNorm2d(512)
        self.intermediate_relu = nn.ReLU(inplace=True)

        # Last two layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        # First layer
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Minimal Intermediate layer
        x = self.intermediate_conv(x)
        x = self.intermediate_bn(x)
        x = self.intermediate_relu(x)

        # Last two layers
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


In [ ]:
#| export
@torch.no_grad()
def get_resnet_encoder(model,n_in=3):
    model = create_body(model, n_in=n_in, pretrained=False, cut=len(list(model.children()))-1)
    model.add_module('flatten', torch.nn.Flatten())
    return model


@torch.no_grad()
def resnet_arch_to_encoder(arch: Literal['smallres','resnet18', 'resnet34', 'resnet50'],
                           weight_type: Literal['random', 'partialtrained', 'imgnet_bt_pretrained', 'imgnet_sup_pretrained'] = 'random'):
    """Given a ResNet architecture, return the encoder configured for 3 input channels.
       The 'weight_type' argument specifies the weight initialization strategy.

    Args:
        arch (Literal['smallres','resnet18', 'resnet34', 'resnet50']): The architecture of the ResNet.
        weight_type (Literal['random', 'partialtrained', 'imgnet_bt_pretrained', 'imgnet_sup_pretrained']): Specifies the weight initialization strategy. Defaults to 'random'.

    Returns:
        Encoder: An encoder configured for 3 input channels and specified architecture.
    """
    
    n_in=3


    if weight_type == 'imgnet_bt_pretrained': test_eq(arch,'resnet50')

    #This means we will load the state_dict from a path.
    if weight_type == 'partialtrained': weight_type = 'random'

    
    if arch == 'resnet50':

        if weight_type == 'imgnet_bt_pretrained':
            _model = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')

        elif weight_type == 'imgnet_sup_pretrained':
            _model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

        elif weight_type == 'random':
            _model = resnet50()
        

    elif arch == 'resnet34':

        if weight_type == 'imgnet_sup_pretrained':
            _model = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)

        elif weight_type == 'random':
            _model = resnet34() 

    elif arch == 'resnet18':
        if weight_type == 'imgnet_sup_pretrained':
            _model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1) 

        elif weight_type == 'random':
            _model = resnet18()

    elif arch == 'smallres':
        _model = _SmallRes()
    
        
    else: raise ValueError('Architecture not recognized')

    return get_resnet_encoder(_model,n_in) 



In [ ]:
#| export

def generate_config_hash(config):
    """
    Generates a unique hash for a given experiment configuration.
    
    Args:
    config (dict or Namespace): Experiment configuration. Can be a dictionary or a namespace object.
    
    Returns:
    str: A unique hash representing the experiment configuration.
    """
    # Convert config to dict if it's a Namespace
    config_dict = vars(config) if not isinstance(config, dict) else config
    
    # Serialize configuration to a sorted JSON string to ensure consistency
    config_str = json.dumps(config_dict, sort_keys=True)
    
    # Generate SHA-256 hash from the serialized string
    hash_obj = hashlib.sha256(config_str.encode())  # Encode to convert string to bytes
    config_hash = hash_obj.hexdigest()
    
    # Optionally, return a truncated version of the hash for readability
    short_hash = config_hash[:8]  # Use the first 8 characters as an example
    return short_hash


Test `generate_config_hash`

In [ ]:
config1 = SimpleNamespace(arch='resnet18', dataset='cifar10', n_in=3, encoder_dimension=512)
config2 = SimpleNamespace(arch='resnet34', dataset='cifar10', n_in=3, encoder_dimension=512)
config3 = SimpleNamespace(arch='resnet50', dataset='cifar10', n_in=3, encoder_dimension=2048)

test_eq(generate_config_hash(config1), generate_config_hash(config1))
test_ne(generate_config_hash(config1), generate_config_hash(config2))
test_ne(generate_config_hash(config1), generate_config_hash(config3))
test_ne(generate_config_hash(config2), generate_config_hash(config3))

config4 = SimpleNamespace(dataset='cifar10', arch='resnet18', n_in=3, encoder_dimension=512)  # Different order
test_eq(generate_config_hash(config1), generate_config_hash(config4))


In [ ]:
#| export
def create_experiment_directory(base_dir, config):
    # Generate a unique hash for the configuration
    unique_hash = generate_config_hash(config)
    
    # Construct the directory path for this experiment
    experiment_dir = os.path.join(base_dir, config.train_type, config.dataset, config.arch, unique_hash)
    
    # Create the directory if it doesn't exist
    os.makedirs(experiment_dir, exist_ok=True)
    
    return experiment_dir,unique_hash


def save_configuration(config, experiment_dir):
    """
    Saves the experiment configuration as a YAML file in the experiment directory.

    Args:
    config (dict, Namespace, or any serializable object): Experiment configuration.
    experiment_dir (str): Path to the directory where the config file will be saved.
    """
    config_file_path = os.path.join(experiment_dir, 'config.yaml')
    
    # Check if config is not a dictionary (e.g., a Namespace object) and convert if necessary
    config_dict = vars(config) if not isinstance(config, dict) else config
    
    with open(config_file_path, 'w') as file:
        yaml.dump(config_dict, file)
    
    print(f"Configuration saved to {config_file_path}")




def save_metadata_file(experiment_dir, git_commit_hash):
    """
    Saves a metadata file with the Git commit hash
    """
    metadata_file_path = os.path.join(experiment_dir, 'metadata.yaml')
    metadata_content = {
        "Git Commit Hash": git_commit_hash,
    }

    with open(metadata_file_path, 'w') as file:
        yaml.dump(metadata_content, file)

    print(f"Metadata saved to {metadata_file_path}")


def update_experiment_index(project_root, details):
    central_json_path = os.path.join(project_root, 'experiment_index.json')
    
    if os.path.exists(central_json_path):
        with open(central_json_path, 'r') as file:
            experiments_index = json.load(file)
    else:
        experiments_index = {}
    
    experiment_hash = details["experiment_hash"]
    experiments_index[experiment_hash] = details
    
    with open(central_json_path, 'w') as file:
        json.dump(experiments_index, file, indent=4)
    
    print(f"Updated experiment index for hash: {experiment_hash}")


def get_latest_commit_hash(repo_path):
    try:
        commit_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=repo_path).decode('ascii').strip()
        return commit_hash
    except subprocess.CalledProcessError as e:
        print(f"Error obtaining latest commit hash: {e}")
        return None

def setup_experiment(config,base_dir):

    # Create a unique directory for this experiment based on its configuration
    # This directory will contain all artifacts related to the experiment, such as model checkpoints and logs.
    experiment_dir, experiment_hash = create_experiment_directory(base_dir, config)

    print(f"The experiment_dir is: {experiment_dir} and the experiment hash is: {experiment_hash}")

    # Save the loaded configuration to the experiment directory as a YAML file
    # This ensures that we can reproduce or analyze the experiment later.
    save_configuration(config, experiment_dir)

    git_commit_hash = get_latest_commit_hash('.')
    print(f"The git hash is: {git_commit_hash}")

    return experiment_dir, experiment_hash,git_commit_hash


In [ ]:
#| export

class InterruptCallback(Callback):
    def __init__(self, interrupt_epoch):
        super().__init__()
        self.interrupt_epoch = interrupt_epoch

    def before_epoch(self):
        if self.epoch == self.interrupt_epoch:
            print(f"Interrupting training before starting epoch {self.interrupt_epoch}")
            raise CancelFitException

class SaveLearnerCheckpoint(Callback):
    def __init__(self, experiment_dir,start_epoch=0, save_interval=250, with_opt=True):
        self.experiment_dir = experiment_dir
        self.start_epoch = start_epoch
        self.save_interval = save_interval
        self.with_opt = with_opt  # Decide whether to save optimizer state as well.

    def after_epoch(self):
        if (self.epoch+1) % self.save_interval == 0 and self.epoch>=self.start_epoch:
            print(f"Saving model and learner state at epoch {self.epoch}")
   
            checkpoint_filename = f"learner_checkpoint_epoch_{self.epoch}"
            checkpoint_path = os.path.join(self.experiment_dir, checkpoint_filename)
            # Save the entire learner object, including the model's parameters and optimizer state.
            self.learn.save(checkpoint_path, with_opt=self.with_opt)
            print(f"Checkpoint saved to {checkpoint_path}")


In [ ]:
save_interval=6
for epoch in range(12):

    if (epoch+1) % save_interval == 0:

        print(f'saving at epoch {epoch}')



saving at epoch 5
saving at epoch 11


In [ ]:
#| export

def extract_epoch(filename):
    """Extract the epoch number from a filename."""
    pattern = re.compile(r"_epoch_(\d+)\.pt[h]?")
    match = pattern.search(filename)
    return int(match.group(1)) if match else None

def find_largest_epoch_file(directory_path):
    """Find the file with the largest epoch number in a directory."""
    max_epoch = -1
    largest_epoch_file = None

    for filename in os.listdir(directory_path):
        epoch = extract_epoch(filename)
        if epoch is not None and epoch > max_epoch:
            max_epoch = epoch
            largest_epoch_file = filename

    return largest_epoch_file

def return_max_filename(filename1, filename2):
    # Improved handling for initial cases
    if not filename1:
        return filename2
    if not filename2:
        return filename1

    # Extract epochs and compare
    epoch1 = extract_epoch(filename1)
    epoch2 = extract_epoch(filename2)

    # Return the filename with the larger epoch number
    return filename1 if epoch1 >= epoch2 else filename2


def get_highest_epoch_path(base_dir, config):
    """
    Check in all experiment directories derived from the config and return the path
    to the file with the highest epoch along with its experiment directory.
    """

    experiment_index_path = base_dir + '/experiment_index.json'

    try: 
        # Load the JSON data from the file
        with open(experiment_index_path, 'r') as file:
            experiment_index = json.load(file)

    except FileNotFoundError:
        return None,None


    # Build the main part of the experiment directory from base_dir and config
    _experiment_dir, _ = create_experiment_directory(base_dir, config)
    base_experiment_dir = os.path.dirname(_experiment_dir)  # Strip the hash part

    print(f"looking in {base_experiment_dir} for highest epoch saved")

    _max_file_path = None
    _max_experiment_dir = None  # To keep track of the directory of the max file

    for k in list(experiment_index.keys()):
        _experiment_dir = experiment_index[k]['experiment_dir']
        if base_experiment_dir not in _experiment_dir:
            continue
        _x = find_largest_epoch_file(_experiment_dir)
        if _x:
            _x_path = os.path.join(_experiment_dir, _x)
            # Update max_file_path and the corresponding experiment_dir if a new max is found
            if not _max_file_path or return_max_filename(_x_path, _max_file_path) == _x_path:
                _max_file_path = _x_path
                _max_experiment_dir = _experiment_dir
    

    print(f"Found max file path: {_max_file_path} and max experiment dir: {_max_experiment_dir}")
    
    return _max_file_path.split('.')[0], _max_experiment_dir  # Return both file path and directory



def get_experiment_state(config,base_dir):
    """Get the load_learner_path, learn_type, start_epoch, interrupt_epoch for BT experiment.
       Basically this tells us how to continue learning (e.g. we have run two sessions for 
       100 epochs, and want to continue for another 100 epochs). Return values are
       None if we are starting from scratch.
    """

    load_learner_path, _  = get_highest_epoch_path(base_dir, config)

    #TODO:
    #We can get start_epoch, interrupt epoch from `get_highest_epoch_path` + save_interval (may be None!)
    start_epoch=0 if load_learner_path is None else int(load_learner_path.split('_')[-1])+1
    interrupt_epoch = start_epoch + config.save_interval

    #We can also get the learn_type from the load_learner_path + weight_type. 
    
    if config.weight_type == 'random':
        learn_type = 'standard'
    
    elif 'pretrained' in config.weight_type:
        learn_type = 'transfer_learning'

    learn_type = learn_type if load_learner_path is None else 'continue_learning'

    return load_learner_path, learn_type, start_epoch, interrupt_epoch
    

In [ ]:

import nbdev; nbdev.nbdev_export()